In [2]:
import numpy as np
import pygame
import sys
import math
import random

#deklarasi variabel yang diperlukan
JML_BARIS = 6
JML_KOLOM = 7
KOTAK = 75
PLAYER = 1
AI = 2

#deklarasi warna yang digunakan
BLUE = (0,0,225)
BLACK = (0,0,0)
RED = (225,0,0)
YELLOW = (255,225,0)

#fungsi untuk membuat papan permainan
def buat_papan():
    papan = np.zeros((JML_BARIS, JML_KOLOM), dtype=int)
    return papan

#fungsi untuk menentukan posisi setiap bagian yg dijatuhkan dalam papan
def isi_papan(papan, baris,kolom, isi):
    papan[baris][kolom] = isi

#fungsi untuk memeriksa jika kolom sudah penuh
def cek_penuh(papan, kolom):
    return papan[JML_BARIS-1][kolom] == 0

#fungsi untuk memeriksa apakah baris sudah terisi belum
def cek_visited(papan, kolom):
    for r in range(JML_BARIS):
        if papan[r][kolom] == 0:
            return r
        
#mencetak papan dengan index (0,0) berada pada kiri bawah
def cetak_papan(papan):
    print(np.flip(papan,0))
        
#fungsi untuk memeriksa jika menang
def cek_menang(papan, isi):
    #cek horizontal
    for c in range (JML_KOLOM-3):
        for r in range (JML_BARIS):
            if papan[r][c] == isi and papan[r][c+1] == papan[r][c+2] == papan[r][c+3] == isi:
                return True    
    #cek vertikal
    for c in range (JML_KOLOM):
        for r in range (JML_BARIS-3):
            if papan[r][c] == isi and papan[r+1][c] == papan[r+2][c] == papan[r+3][c] == isi:
                return True        
    #cek diagonal positif
    for c in range (JML_KOLOM-3):
        for r in range (JML_BARIS-3):
            if papan[r][c] == isi and papan[r+1][c+1] == papan[r+2][c+2] == papan[r+3][c+3] == isi:
                return True 
    #cek diagonal negatif
    for c in range (JML_KOLOM-3):
        for r in range (3,JML_BARIS):
            if papan[r][c] == isi and papan[r-1][c+1] == papan[r-2][c+2] == papan[r-3][c+3] == isi:
                return True 

#Mulai Ai

def nilai_pemain(window, isi):
	score = 0
	isi_lawan = PLAYER
	if isi == PLAYER:
		isi_lawan = AI
    

	if window.count(isi) == 4:
	    score += 100
	elif window.count(isi) == 3 and window.count(0) == 1:
	    score += 5
	elif window.count(isi) == 2 and window.count(0) == 2:
	    score += 2
	if window.count(isi_lawan) == 3 and window.count(0) == 1:
		score -= 4

	return score


def cek_nilai(papan, isi):

    score = 0

    #score horizontal
    for r in range(JML_BARIS):
        array_baris = [int (i) for i in list(papan[r,:])]
        #cek sampai ada empat isi tiap baris 
        for c in range(JML_KOLOM-3):
            window = array_baris[c:c+4]
            score += nilai_pemain(window,isi)
    #score vertikal
    for c in range(JML_KOLOM):
        array_kolom = [int (i) for i in list(papan[:,c])]
        for r in range(JML_BARIS-3):
            window = array_kolom[r:r+4]
            score += nilai_pemain(window,isi)
    #score diagonal
    for r in range(JML_BARIS-3):
        for c in range(JML_KOLOM-3):
            window = [papan[r+i][c+i] for i in range(4)]
            score += nilai_pemain(window, isi)
    for r in range(JML_BARIS-3):
        for c in range(JML_KOLOM-3):
            window = [papan[r+3-i][c+i] for i in range(4)]
            score += nilai_pemain(window, isi)
    return score
    #score tengah kolom
    center_array = [int(i) for i in list(papan[:, JML_KOLOM//2])]
    center_count = center_array.count(isi)
    score += center_count*3

def get_cek_penuh(papan):
    cek = []
    for kolom in range(JML_KOLOM):
        if cek_penuh(papan,kolom):
            cek.append(kolom)
    return cek

def best_move (papan, isi):
    
    cek = get_cek_penuh(papan)
    best_score = 0
    best_kolom = random.choice(cek)
    for kolom in cek:
        baris = cek_visited(papan, kolom)
        temp_papan = papan.copy()
        isi_papan(temp_papan, baris, kolom, isi)
        score = cek_nilai(temp_papan, isi)
        if score > best_score:
            best_score = score
            best_kolom = kolom
    return best_kolom

#fungsi untuk menampilkan dengan pygame        
def draw_board(papan):
    for c in range(JML_KOLOM):
        for r in range (JML_BARIS):
            pygame.draw.rect(screen, BLUE,(c*KOTAK,r*KOTAK+KOTAK, KOTAK, KOTAK))
            pygame.draw.circle(screen, BLACK,(int(c*KOTAK+KOTAK/2),int(r*KOTAK+KOTAK+KOTAK/2)), radius )
    for c in range(JML_KOLOM):
        for r in range (JML_BARIS):
            if papan[r][c] == PLAYER:
                pygame.draw.circle(screen, RED,(int(c*KOTAK+KOTAK/2),int(tinggi-(r*KOTAK+KOTAK/2)) ), radius )
            elif papan[r][c] == AI:
                pygame.draw.circle(screen, YELLOW,(int(c*KOTAK+KOTAK/2),int( tinggi - (r*KOTAK+KOTAK/2) )), radius )
    pygame.display.update()
    

papan = buat_papan()
cetak_papan(papan)
game_over = False
turn = 0
pygame.init()
lebar = JML_KOLOM * KOTAK
tinggi = (JML_BARIS+1)* KOTAK 
size = (lebar,tinggi)
radius = int (KOTAK/2 - 5 )
screen = pygame.display.set_mode(size)
draw_board(papan)
pygame.display.update()
tulisan = pygame.font.SysFont("Arial",50)


while not game_over :
    for event in pygame.event.get():

        #untuk keluar dari tampilan
        if event.type == pygame.QUIT:
            sys.exit()

        #yang ditampilkan pada gerakan mouse
        if event.type == pygame.MOUSEMOTION:
            pygame.draw.rect(screen, BLACK,(0,0, lebar, KOTAK))
            posx = event.pos[0]
            if turn == 0:
                pygame.draw.circle(screen, RED, (posx, int(KOTAK/2)), radius )
        pygame.display.update()


        if event.type == pygame.MOUSEBUTTONDOWN:
            pygame.draw.rect(screen, BLACK,(0,0, lebar, KOTAK))

            if turn == 0:
                posx = event.pos[0]
                kolom = int(math.floor(posx/KOTAK))          
                if cek_penuh(papan, kolom):
                    baris = cek_visited(papan,kolom)
                    isi_papan(papan, baris,kolom, PLAYER)
                    if cek_menang(papan, PLAYER):
                        label = tulisan.render("Pemain menang!!!", 1, RED )
                        screen.blit(label,(40,10))
                        game_over = True
                    turn += 1
                    turn = turn%2
                    cetak_papan(papan)
                    draw_board(papan)
                else:
                    pygame.quit()
    
    if turn == 1 and not game_over:  
        #kolom = random.randint(0,JML_KOLOM-1)
        kolom = best_move(papan, AI)
        if cek_penuh(papan, kolom):
            baris = cek_visited(papan,kolom)
            isi_papan(papan, baris,kolom, AI)
            if cek_menang(papan, AI):
                label = tulisan.render("Komputer menang!!!", 1, YELLOW )
                screen.blit(label,(40,10))
                game_over = True
                   
            cetak_papan(papan)
            draw_board(papan)
            turn += 1
            turn = turn%2
            
    if game_over:
        pygame.time.wait(3000)
        pygame.quit()

[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0]]
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 2 1 0 0]]
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 2 1 1 0]]
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [2 0 0 2 1 1 0]]
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0]
 [2 0 0 2 1 1 0]]
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0]
 [2 2 0 2 1 1 0]]
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0]
 [2 2 1 2 1 1 0]]
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 2 0 0 1 0 0]
 [2 2 1 2 1 1 0]]
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 2